In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
import sys
import cv2
import os
import math
import pickle
import copy
import numpy as np
sys.path.append("C:\\Users\\user\\Documents\\Workspace\\MeronymNet-PyTorch\\src")

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.utils import dense_to_sparse, to_dense_adj
import torch_geometric.nn as gnn
from torch_geometric.data import DataLoader
import torchvision
import torch.utils.data as data_utils
from torch.utils.tensorboard import SummaryWriter

In [19]:
from losses import BoxVAE_losses as loss
from components.AutoEncoder import GCNAutoEncoder
from components.Decoder import Decoder

In [8]:
import matplotlib.pyplot as plt

In [9]:
colors = [(1, 0, 0),
          (0.737, 0.561, 0.561),
          (0.255, 0.412, 0.882),
          (0.545, 0.271, 0.0745),
          (0.98, 0.502, 0.447),
          (0.98, 0.643, 0.376),
          (0.18, 0.545, 0.341),
          (0.502, 0, 0.502),
          (0.627, 0.322, 0.176),
          (0.753, 0.753, 0.753),
          (0.529, 0.808, 0.922),
          (0.416, 0.353, 0.804),
          (0.439, 0.502, 0.565),
          (0.784, 0.302, 0.565),
          (0.867, 0.627, 0.867),
          (0, 1, 0.498),
          (0.275, 0.51, 0.706),
          (0.824, 0.706, 0.549),
          (0, 0.502, 0.502),
          (0.847, 0.749, 0.847),
          (1, 0.388, 0.278),
          (0.251, 0.878, 0.816),
          (0.933, 0.51, 0.933),
          (0.961, 0.871, 0.702)]
colors = (np.asarray(colors)*255)
canvas_size = 550
def plot_bbx(bbx):
    bbx = bbx*canvas_size
    canvas = np.ones((canvas_size,canvas_size,3), np.uint8) * 255
    for i, coord in enumerate(bbx):
        x_minp, y_minp,x_maxp , y_maxp= coord
        if [x_minp, y_minp,x_maxp , y_maxp]!=[0,0,0,0]:
            cv2.rectangle(canvas, (int(x_minp), int(y_minp)), (int(x_maxp) , int(y_maxp) ), colors[i], 6)
    return canvas


In [10]:
def inference(decoder, nodes, obj_class, latent_dims, batch_size):

    decoder.cuda()
    z_latent = torch.normal(torch.zeros([batch_size,latent_dims])).cuda()
    nodes = torch.reshape(nodes,(batch_size,decoder.num_nodes))
    obj_class = torch.reshape(obj_class,(batch_size, decoder.class_size))
    conditioned_z = torch.cat([nodes, z_latent],dim=-1)
    conditioned_z = torch.cat([obj_class, conditioned_z],dim=-1)
    
    x_bbx, x_lbl, x_edge, class_pred = decoder(conditioned_z)
        
    return x_bbx, x_lbl, x_edge, class_pred

In [11]:
outfile = 'D:/meronym_data/X_train.np'
with open(outfile, 'rb') as pickle_file:
    X_train = pickle.load(pickle_file)

outfile = 'D:/meronym_data/X_train_val.np'
with open(outfile, 'rb') as pickle_file:
    X_train_val = pickle.load(pickle_file)

outfile = 'D:/meronym_data/class_v.np'
with open(outfile, 'rb') as pickle_file:
    class_v = pickle.load(pickle_file)

outfile = 'D:/meronym_data/class_v_val.np'
with open(outfile, 'rb') as pickle_file:
    class_v_val = pickle.load(pickle_file)

outfile = 'D:/meronym_data/adj_train.np'
with open(outfile, 'rb') as pickle_file:
    adj_train = pickle.load(pickle_file)

outfile = 'D:/meronym_data/adj_train_val.np'
with open(outfile, 'rb') as pickle_file:
    adj_train_val = pickle.load(pickle_file)

In [12]:
train_idx = np.random.randint(1,len(X_train),80000)
val_idx = np.random.randint(1,len(X_train_val),256)

In [15]:
batch_size = 64
train_list =[]
for idx, batch in enumerate(zip(copy.deepcopy(X_train[train_idx]),
                                copy.deepcopy(class_v[train_idx]),
                                copy.deepcopy(adj_train[train_idx]))):
    edge_index, _ = dense_to_sparse(torch.from_numpy(batch[2]).float())
    train_list.append(Data(x = torch.from_numpy(batch[0]).float(),
                           y = torch.from_numpy(batch[1]).float(),
                           edge_index = edge_index
                                )
                     )

batch_train_loader = DataLoader(train_list, batch_size=batch_size, pin_memory=True)

val_list = []
for idx, batch in enumerate(zip(copy.deepcopy(X_train_val[val_idx]),
                                copy.deepcopy(class_v_val[val_idx]), 
                                copy.deepcopy(adj_train_val[val_idx]))):
    edge_index, _ = dense_to_sparse(torch.from_numpy(batch[2]).float())
    val_list.append(Data(x = torch.from_numpy(batch[0]).float(),
                         y = torch.from_numpy(batch[1]).float(),
                         edge_index = edge_index
                                )
                     )
batch_val_loader = DataLoader(val_list, batch_size=batch_size, pin_memory=True)

C:\Users\user\miniconda3\lib\site-packages\torch_geometric\deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [16]:
idx = 0
for data in batch_train_loader:
    idx+=1
    print(data.x.shape,data.y.shape,data.edge_index.shape)
    if idx==3:
        break

torch.Size([1536, 5]) torch.Size([640]) torch.Size([2, 2844])
torch.Size([1536, 5]) torch.Size([640]) torch.Size([2, 2930])
torch.Size([1536, 5]) torch.Size([640]) torch.Size([2, 3034])


In [13]:
latent_dims = 64
batch_size = 64
num_nodes = 24
bbx_size = 4
num_classes = 10
label_shape = 1
nb_epochs = 200
klw = losses.frange_cycle_linear(nb_epochs)
learning_rate = 0.000062
hidden1 = 32
hidden2 = 16
hidden3 = 128


In [28]:
reconstruction_loss_arr = []
kl_loss_arr = []
bbox_loss_arr = []
adj_loss_arr = []
node_loss_arr = []

reconstruction_loss_val_arr = []
kl_loss_val_arr = []
bbox_loss_val_arr = []
adj_loss_val_arr = []
node_loss_val_arr = []

vae = GCNAutoEncoder(latent_dims,num_nodes,bbx_size,num_classes,label_shape,hidden1, hidden2, hidden3)
vae.cuda()
optimizer = optim.Adam(vae.parameters(), lr=learning_rate)
model_path = ('D:/meronym_data/model/GCN-lr-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3))
summary_path = ('D:/meronym_data/runs/GCN-lr-'
                        +str(learning_rate)
                        +'-batch-'+str(batch_size)
                        +'-h1-'+str(hidden1)
                        +'-h2-'+str(hidden2)
                        +'-h3-'+str(hidden3))
if not os.path.exists(model_path):
    os.mkdir(model_path)
writer = SummaryWriter(summary_path)
icoef = 0

for epoch in range(nb_epochs):

    batch_loss = torch.tensor([0.0]).cuda()
    batch_kl_loss = torch.tensor([0.0]).cuda()
    batch_bbox_loss = torch.tensor([0.0]).cuda()
    batch_adj_loss = torch.tensor([0.0]).cuda()
    batch_node_loss = torch.tensor([0.0]).cuda()
    images = []
    
    vae.train()
    i=0
    for train_data in batch_train_loader:
        
        if torch.cuda.is_available():
            train_data = train_data.cuda()
        
        node_data_true = train_data.x
        label_true = node_data_true[:,:1]
        class_true = train_data.y
        adj_true = train_data.edge_index
        batch = train_data.batch
        
        for param in vae.parameters():
            param.grad=None
        
        output = vae(adj_true, node_data_true, label_true , class_true)
        node_data_pred, label_pred, adj_pred, class_pred, z_mean, z_logvar = output
        
        kl_loss = loss.kl_loss(z_mean, z_logvar)
        adj_loss = loss.adj_loss(adj_pred, adj_true, batch, num_nodes)
        bbox_loss = loss.bbox_loss(node_data_pred, node_data_true[:,1:])
        node_loss = loss.node_loss(label_pred,label_true)
        class_loss = loss.class_loss(class_pred, class_true)
        
        kl_weight = klw[icoef]
        
        reconstruction_loss = kl_loss*kl_weight + (bbox_loss + node_loss + adj_loss + class_loss)*24*5
        reconstruction_loss.backward()
        optimizer.step()
        i+=1
      
        batch_loss += reconstruction_loss
        batch_kl_loss += kl_loss
        batch_bbox_loss += bbox_loss
        batch_adj_loss += adj_loss
        batch_node_loss += node_loss
    
        if i%200==0:
            print(i)
            global_step = epoch*len(batch_train_loader)+i
            
            writer.add_scalar("Loss/train/reconstruction_loss", batch_loss.detach()/(i+1), global_step)
            writer.add_scalar("Loss/train/kl_loss", batch_kl_loss.detach()/(i+1), global_step)
            writer.add_scalar("Loss/train/bbox_loss", batch_bbox_loss.detach()/(i+1), global_step)
            writer.add_scalar("Loss/train/adjacency_loss", batch_adj_loss.detach()/(i+1), global_step)
            writer.add_scalar("Loss/train/node_loss", batch_node_loss.detach()/(i+1), global_step)
       
    global_step = epoch*len(batch_train_loader)+i
    image_shape = [num_nodes, bbx_size]

    image = plot_bbx(np.reshape((node_data_true[:24,1:]*label_true[:24]).detach().to("cpu").numpy(),
                                image_shape)).astype(float)/255
    writer.add_image('train/images/input', image, global_step, dataformats='HWC')
    image = plot_bbx((node_data_pred[0]*label_true[:24]).detach().to("cpu").numpy()).astype(float)/255
    writer.add_image('train/images/generated', image, global_step, dataformats='HWC')
    
    reconstruction_loss_arr.append(batch_loss.detach()/(i+1))
    kl_loss_arr.append(batch_kl_loss.detach()/(i+1))
    bbox_loss_arr.append(batch_bbox_loss.detach()/(i+1))
    adj_loss_arr.append(batch_adj_loss.detach()/(i+1))
    node_loss_arr.append(batch_node_loss.detach()/(i+1))
    
    
    print('[%d, %5d] loss: %.3f' %
              (epoch + 1, i + 1, batch_loss/(i+1) ))
    
    batch_loss = torch.tensor([0.0]).cuda()
    batch_kl_loss = torch.tensor([0.0]).cuda()
    batch_bbox_loss = torch.tensor([0.0]).cuda()
    batch_adj_loss = torch.tensor([0.0]).cuda()
    batch_node_loss = torch.tensor([0.0]).cuda()
    images = []
    vae.eval()
    for i, val_data in enumerate(batch_val_loader, 0):
        val_data.cuda()
        node_data_true = val_data.x
        label_true = node_data_true[:,:1]
        class_true = val_data.y
        adj_true = val_data.edge_index
        batch = val_data.batch
        
        kl_weight = klw[icoef]
        
        output = vae(adj_true, node_data_true, label_true , class_true)
        node_data_pred, label_pred, adj_pred, class_pred, z_mean, z_logvar = output
        
        kl_loss = losses.kl_loss(z_mean, z_logvar)
        adj_loss = losses.adj_loss(adj_pred, adj_true, batch, num_nodes)
        bbox_loss = losses.bbox_loss(node_data_pred, node_data_true[:,1:])
        node_loss = losses.node_loss(label_pred,label_true)
        class_loss = losses.class_loss(class_pred, class_true)
        
        reconstruction_loss = kl_loss*kl_weight + (bbox_loss + node_loss + adj_loss + class_loss)*24*5
        
        batch_loss += reconstruction_loss
        batch_kl_loss += kl_loss
        batch_bbox_loss += bbox_loss
        batch_adj_loss += adj_loss
        batch_node_loss += node_loss
    
    image = plot_bbx(np.reshape((node_data_true[:24,1:]*label_true[:24]).detach().to("cpu").numpy(),
                                image_shape)).astype(float)/255
    writer.add_image('val/images/input', image, global_step, dataformats='HWC')
    image = plot_bbx((node_data_pred[0]*label_true[:24]).detach().to("cpu").numpy()).astype(float)/255
    writer.add_image('val/images/generated', image, global_step, dataformats='HWC')

    reconstruction_loss_val_arr.append(batch_loss/(i+1))
    kl_loss_val_arr.append(batch_kl_loss/(i+1))
    bbox_loss_val_arr.append(batch_bbox_loss/(i+1))
    adj_loss_val_arr.append(batch_adj_loss/(i+1))
    node_loss_val_arr.append(batch_node_loss/(i+1))
    
    writer.add_scalar("Loss/val/reconstruction_loss", batch_loss.detach()/(i+1), global_step)
    writer.add_scalar("Loss/val/kl_loss", batch_kl_loss.detach()/(i+1), global_step)
    writer.add_scalar("Loss/val/kl_loss", batch_kl_loss.detach()/(i+1), global_step)
    writer.add_scalar("Loss/val/bbox_loss", batch_bbox_loss.detach()/(i+1), global_step)
    writer.add_scalar("Loss/val/adjacency_loss", batch_adj_loss.detach()/(i+1), global_step)
    writer.add_scalar("Loss/val/node_loss", batch_node_loss.detach()/(i+1), global_step)
   
    if epoch%50 == 0:
        torch.save(vae.state_dict(), model_path + '/model_weights.pth')

    if kl_loss_arr[-1]>0.5 and abs(bbox_loss_arr[-1] - bbox_loss_val_arr[-1]) < 0.03:
        icoef = icoef + 1  

torch.save(vae.state_dict(),model_path + '/model_weights.pth')

for i in range(batch_size):    
    image = plot_bbx(np.reshape((node_data_true[:24,1:]*label_true[:24]).detach().to("cpu").numpy(),
                                image_shape)).astype(float)/255
    writer.add_image('result/images/input/'+str(i), image, global_step, dataformats='HWC')
    image = plot_bbx((node_data_pred[i]*label_true[:24]).detach().to("cpu").numpy()).astype(float)/255
    writer.add_image('result/images/generated/'+str(i), image, global_step, dataformats='HWC')
    
writer.flush()
writer.close()
print('Finished Training')

C:\Users\user\Documents\Workspace\MeronymNet-PyTorch\src\components\Decoder.py:45: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  class_pred = self.act2(self.dense_cls(x))


200
400
600
800
1000
1200
[1,  1251] loss: 403.892
200
400
600
800
1000
1200
[2,  1251] loss: 384.969
200
400
600
800
1000
1200
[3,  1251] loss: 383.019
200
400
600
800
1000
1200
[4,  1251] loss: 379.457
200
400
600
800
1000
1200
[5,  1251] loss: 369.378
200
400
600
800
1000
1200
[6,  1251] loss: 357.129
200
400
600
800
1000
1200
[7,  1251] loss: 348.868
200
400
600
800
1000
1200
[8,  1251] loss: 334.915
200
400
600
800
1000
1200
[9,  1251] loss: 316.893
200
400
600
800
1000
1200
[10,  1251] loss: 306.343
200
400
600
800
1000
1200
[11,  1251] loss: 296.794
200
400
600
800
1000
1200
[12,  1251] loss: 289.017
200
400
600
800
1000
1200
[13,  1251] loss: 285.408
200
400
600
800
1000
1200
[14,  1251] loss: 282.992
200
400
600
800
1000
1200
[15,  1251] loss: 277.049
200
400
600
800
1000
1200
[16,  1251] loss: 270.850
200
400
600
800
1000
1200
[17,  1251] loss: 267.702
200
400
600
800
1000
1200
[18,  1251] loss: 264.545
200
400
600
800
1000
1200
[19,  1251] loss: 257.838
200
400
600
800
1000


200
400
600
800
1000
1200
[158,  1251] loss: 199.496
200
400
600
800
1000
1200
[159,  1251] loss: 199.423
200
400
600
800
1000
1200
[160,  1251] loss: 199.320
200
400
600
800
1000
1200
[161,  1251] loss: 199.308
200
400
600
800
1000
1200
[162,  1251] loss: 199.239
200
400
600
800
1000
1200
[163,  1251] loss: 199.156
200
400
600
800
1000
1200
[164,  1251] loss: 199.134
200
400
600
800
1000
1200
[165,  1251] loss: 199.054
200
400
600
800
1000
1200
[166,  1251] loss: 198.974
200
400
600
800
1000
1200
[167,  1251] loss: 198.925
200
400
600
800
1000
1200
[168,  1251] loss: 198.878
200
400
600
800
1000
1200
[169,  1251] loss: 198.844
200
400
600
800
1000
1200
[170,  1251] loss: 198.746
200
400
600
800
1000
1200
[171,  1251] loss: 198.689
200
400
600
800
1000
1200
[172,  1251] loss: 198.664
200
400
600
800
1000
1200
[173,  1251] loss: 198.600
200
400
600
800
1000
1200
[174,  1251] loss: 198.561
200
400
600
800
1000
1200
[175,  1251] loss: 198.516
200
400
600
800
1000
1200
[176,  1251] loss: 1

In [36]:
#testing loop
writer = SummaryWriter(summary_path)

vae = AutoEncoder(latent_dims,num_nodes,bbx_size,num_classes,label_shape,hidden1, hidden2, hidden3)
vae.load_state_dict(torch.load(model_path+ '/model_weights.pth'))

decoder = vae.decoder
image_shape = [num_nodes, bbx_size]
global_step = 250000
for i, val_data in enumerate(batch_val_loader, 0):
    
    val_data.cuda()
    node_data_true = val_data.x
    label_true = node_data_true[:,:1]
    class_true = val_data.y
    output = inference(decoder, label_true , class_true, latent_dims, batch_size)
    node_data_pred_test, _, _, _ = output
    
    break

for i in range(batch_size):    
    image = plot_bbx(np.reshape((node_data_true[24*i:24*(i+1),1:]*label_true[24*i:24*(i+1)]).detach().to("cpu").numpy(),
                                image_shape)).astype(float)/255
    writer.add_image('test_result/images/'+str(i)+'/input/', image, global_step, dataformats='HWC')
    image = plot_bbx((node_data_pred_test[i]*label_true[24*i:24*(i+1)]).detach().to("cpu").numpy()).astype(float)/255
    writer.add_image('test_result/images/'+str(i)+'/generated/', image, global_step, dataformats='HWC')

writer.flush()
writer.close()